In [1]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import os

In [34]:
train_samples = ['409', '418', '350', '399', '361', '430', '380', '359', '371', '377', '379', '360', '368', '419', '389', '420', '401', '408', '352', '388', '362', '421', '412', '351', '349', '390', '400', '378']
test_samples = ['411', '387', '410', '398', '370', '369', '397']

In [135]:
palette = {
    0 : (0, 128, 0),  #poseidonia
    1 : (0, 0, 255),  #rock
    2 : (255, 0, 0),  #macroalgae
    3 : (255, 128, 0),  #sand
    4 : (0, 0, 0)  # Undefined (black)
}
invert_palette = {v: k for k, v in palette.items()}
def convert_from_color(arr_3d, palette=invert_palette):
    """RGB-color encoding to grayscale labels"""
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.uint8)

    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    return arr_2d

def extract_features(sample, r=2):
    tx_train = []
    ty_train = []
    with rasterio.open(f"../data/benchmark-datasets/MagicBathyNet/agia_napa/gts/s2/gts_{sample}.tif") as f:
        mask = f.read().transpose(1,2,0)
        mask = convert_from_color(mask)

    with rasterio.open(f"../data/benchmark-datasets/MagicBathyNet/agia_napa/img/s2/img_{sample}.tif") as f:
        img = f.read().transpose(1,2,0)
    
    # pad mask with "4" on each side by 1 pixel
    mask = np.pad(mask, r, 'constant', constant_values=4)
    # pad img with zeros on each side by 1 pixel
    img = np.pad(img, ((r,r), (r,r), (0,0)), 'constant', constant_values=0)

    xs, ys = np.where(mask!=4)
    
    for x, y in zip(xs, ys):
        feature = img[x-r:x+r+1, y-r:y+r+1].flatten()
        tx_train.append(feature)
        ty_train.append(mask[x, y])
        
    return np.array(tx_train), np.array(ty_train)


x_train = []
y_train = []
for sample in train_samples:
    x, y = extract_features(sample)
    x_train.append(x)
    y_train.append(y)
x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)

x_test = []
y_test = []
for sample in test_samples:
    x, y = extract_features(sample)
    x_test.append(x)
    y_test.append(y)
x_test = np.concatenate(x_test)
y_test = np.concatenate(y_test)

In [136]:
x_train.shape, x_test.shape

((541, 75), (147, 75))

In [137]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [142]:
model = RandomForestClassifier(n_estimators=500, criterion="entropy", max_depth=None, n_jobs=-1, class_weight=None)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1_score(y_test, y_pred, average=None), np.mean(f1_score(y_test, y_pred, average=None))

(array([0.86075949, 0.7826087 , 0.5       , 0.75      ]), 0.723342047330765)

In [75]:
(84.21 + 25.00 + 90.70 + 74.78) / 4

68.6725